# Предсказываем стоимость мед страховки



## Загужаем необходимые библиотеки

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Загружаем данные

Собраны данные:
- age: возраст
- sex: пол
- bmi: индекс массы тела
- children: количество детей, охваченных медицинским страхованием / количество иждивенцев
- smoker: курение
- region: регион (northeast, southeast, southwest, northwest).
- charges: индивидуальные медицинские расходы (его и хотим предсказать)

In [ ]:
df=pd.read_csv('insurance.csv', sep=',')

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Columns: 712 entries, <!DOCTYPE html><html><head><meta name="google" content="notranslate"><meta http-equiv="X-UA-Compatible" content="IE=edge;"><style nonce="GgnW7B4oDweN3oMyXPrDzw">@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:300;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmSU5fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:700;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmWUlfBBc9.ttf)format('truetype');}</style><meta name="referrer" content="origin"><title>insurance.csv - Google Drive</title><meta property="og:title" content="i

In [ ]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Преобразуем строковые данные

In [ ]:
# Заменяем пол и курение на числа
df['sex']=df['sex'].map({'male':1, 'female':0})
df['smoker']=df['smoker'].map({'yes':1,'no':0})

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [ ]:
# Заменяем регион на набор отдельных колонок (is_southwest, is_southeast и тп)
df = pd.get_dummies(df, columns=['region'])
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


## Формируем признаки и целевую переменную

In [ ]:
## Сформируйте признаки и целевую переменную
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges',
       'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest'],
      dtype='object')

In [ ]:
X=df[['age','bmi','sex','children','smoker','region_northeast','region_northwest','region_southeast','region_southwest']]
y=df['charges']
X

,age,bmi,sex,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,0,1,0,0,0,1
1,18,33.770,1,1,0,0,0,1,0
2,28,33.000,1,3,0,0,0,1,0
3,33,22.705,1,0,0,0,1,0,0
4,32,28.880,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,30.970,1,3,0,0,1,0,0
1334,18,31.920,0,0,0,1,0,0,0
1335,18,36.850,0,0,0,0,0,1,0
1336,21,25.800,0,0,0,0,0,0,1


In [ ]:
y

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64

## Разделяем данные на выборку для обучения/проверки

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
print(X_train, X_test, y_train, y_test)

      age     bmi  sex  ...  region_northwest  region_southeast  region_southwest
1256   51  36.385    0  ...                 1                 0                 0
264    53  38.060    0  ...                 0                 1                 0
318    44  27.645    0  ...                 1                 0                 0
323    57  40.945    1  ...                 0                 0                 0
211    40  30.875    1  ...                 1                 0                 0
...   ...     ...  ...  ...               ...               ...               ...
637    35  38.095    0  ...                 0                 0                 0
322    34  30.800    1  ...                 0                 0                 1
260    58  25.200    0  ...                 0                 0                 1
314    27  31.400    0  ...                 0                 0                 1
1046   43  25.080    0  ...                 0                 0                 0

[1070 rows x 9 

## Создаем и обучаем модель линейной регресии

In [ ]:
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)# Создайте и обучите модель

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
lr=LinearRegression()

In [ ]:
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
lr.coef_

array([  249.44177915,   327.69153228,    44.60848241,   607.67414276,
       23853.28594916,   427.23419219,   303.2141223 ,  -342.24013796,
        -388.20817653])

## Получаем предсказание и оцениваем качество

In [ ]:
lr.predict(X_test)# Получите предсказание

array([ 8266.47159808,  6327.58894084,  8453.0956032 , 11598.08828462,
       29983.39557214, 31902.91728946, 13641.72019062,  3789.65395312,
        4472.777557  , 31900.8922332 ,   970.80367039,  9556.30790314,
       35504.32454033, 12174.61535285,  1249.72877206, 10461.25483182,
        3622.23042365,  4644.8491641 , 10254.53639863, 10038.15658711,
        8974.92877123, 11509.10069431, 35638.96174289, 11238.55944945,
       37364.06401071,  9684.62527101, 37420.76686143,  9844.67830797,
       12876.53652612, 11473.50514021,  6303.82860312, 36811.31948117,
        1223.86289916, 13361.66680726, 11742.64126331,  5673.73037259,
       13483.30958945, 13797.69026612,   917.10119094, 37214.5565418 ,
        8922.88728172, 14838.91007328,  5599.45873996,  8095.3681044 ,
        8979.25420866,  4604.62178074, 30599.85544093, 13623.67106728,
        9744.88069934, 29755.56619771, 10910.21227759, 10610.63872336,
       39191.52471271, 34175.97715715,  8696.66401067, 11086.55301249,
      

In [ ]:
mean_squared_error(y_test, lr.predict(X_test))# Оцените качество, при помощи метода mean_squared_error для тестовой выборки

35457966.449315324

In [ ]:
mean_squared_error(y_train, lr.predict(X_train))

36836757.4562042

## Делаем предсказание для одного человека

In [ ]:
# Заполняем данные по конкретному человеку
data = [{
    "age": 20,
    "sex": 1,
    "bmi": 30,
    "children": 2,
    "smoker": 1,
    "region_northeast": 0,
    "region_northwest": 0,
    "region_southeast": 1,
    "region_southwest": 0
}]

In [ ]:
df_person = pd.DataFrame(data)
df_person.head()

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,20,1,30,2,1,0,0,1,0


In [ ]:
lr.predict(df_person)

array([19294.05372258])

In [ ]:
from sklearn.metrics import roc_auc_score

x = [0,1,1,0,0,1,1,0,0,0]

x_ver = [0.35, 0.85, 0.75, 0.25, 0.05, 0.45, 0.95, 0.65, 0.15, 0.55]

print(f'roc-auc: {roc_auc_score(x, x_ver)}')

roc-auc: 0.9166666666666667
